# ML в Биологии
## ATE

In [ ]:
import pandas as pd
import numpy as np
import scipy.stats as sps

from matplotlib import pyplot as plt
from matplotlib import style
import seaborn as sns

from tqdm.notebook import tqdm

import statsmodels.api as sm
import statsmodels.datasets as smd
import statsmodels.formula.api as smf
import statsmodels.stats.api as sms

import graphviz as gr
from IPython.display import Image

import warnings
warnings.filterwarnings("ignore")

sns.set(style='whitegrid', font_scale=1.3, palette='Set2')

## Учет различных признаков в линейной регрессии для оценки ATE

На основе материалов <a href="https://matheusfacure.github.io/python-causality-handbook/landing-page.html">учебника</a>.

$\newcommand{\indep}{\perp \!\!\! \perp}$
$\newcommand{\notindep}{\not\!\perp \!\!\! \perp}$

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

### 0. Инициализация и обучение моделей через формулы
В этом семинаре нам понадобится модуль `statsmodels.formula.api`, который позволяет задавать модели, указав формулу в виде `target ~ features`, которая означает линейную функцию `target` от `features`.

Рассмотрим пример задания модели линейной регрессии OLS (подробнее можно почитать в [документации](https://www.statsmodels.org/dev/example_formulas.html)).


In [ ]:
df = sm.datasets.get_rdataset("Guerry", "HistData").data
df = df[['Lottery', 'Literacy', 'Wealth', 'Region']].dropna()
df.head()

Итак, предположим, нам нужно обучить модель для таргета `Lottery` по признакам `Literacy`, `Wealth` и `Region` из датасета выше. Тогда синтаксис будет выглядеть таким образом:

In [ ]:
model = smf.ols(formula='Lottery ~ Literacy + Wealth + Region', data=df)
res = model.fit()

Метод `fit` возвращает объект типа `statsmodels.regression.linear_model.RegressionResults`, который содержит различные характеристики модели:
* в первой секции представлены различные характеристики самой модели
  * параметры обучения,
  * $R^2$
  * проверка гипотезы о значимости регрессии в целом
  * логарифмическая функция правдоподобия
  * информационные критерии
* во второй секции &mdash; подробная информация о коэффициентах, включая анализ их стат. значимости:
  * названия коэффициентов (совпадают с именами признаков, или `Intercept` для свободного коэффициента);
  * оценки коэффициентов;
  * стандартные ошибки &mdash; корень из оценки дисперсии оценки каждого из коэффициентов;
  * t-статистика для проверки гипотезы о незначимости коэффициента;
  * p-value критерия для проверки гипотезы о незначимости коэффициента (правило: если $\text{p-value} \leqslant \alpha=0.05$, то гипотеза отвергается, иначе &mdash; не отвергается);
  * левая и правая граница 95\% доверительного интервала для коэффициента.
* в третьей секции &mdash; анализ остатков модели
  * критерии проверки несмещенности, нормальности

*Подробнее:* https://www.statsmodels.org/dev/generated/statsmodels.regression.linear_model.RegressionResults.html

Посмотрим на результат **в предположении гомоскедастичности**

In [ ]:
print(res.summary())

Заметим, что модель автоматически добавила свободный член (`Intercept`) и рассмотрела признак `Region` как категориальный, затем удалив одну из категорий (`С`). Так произошло, потому что значения были строками. При этом, если бы мы имели целочисленный призак и хотели бы рассмотреть его как категориальный, то заключили бы его в `C()` при передаче в формулу. То есть, чтобы специфически указать, что признак категориальный, вот так:


In [ ]:
model = smf.ols(formula='Lottery ~ Literacy + Wealth + C(Region)', data=df)
res = model.fit()
print(res.params)

*Примечение:* Во всех моделях ниже нас будет интересовать коэффициент при параметре, определяющим принадлежность к А или В группе, а также p-value для него.

### 1. Использование признаков: хороший случай

**Задача:** исследовать влияние отправки email с напоминанием о долге. Таргет &mdash; сумма платежей от клиентов, которые просрочили платеж.

Выбраны 5000 случайных клиентов для проведения *рандомизированного эксперимента*. Каждый клиент получает электронное письмо случайно с вероятностью 0.5.

Загрузим данные `collections_email.csv` из [репозитория](https://github.com/matheusfacure/python-causality-handbook/blob/master/causal-inference-for-the-brave-and-true/data/collections_email.csv)

In [ ]:
DATA_PATH = "/content/drive/MyDrive/colab_data/collections_email.csv"

In [ ]:
data = pd.read_csv("/content/collections_email.csv")
data.head()

Поскольку данные рандомизированы, простая разность средних оценивает ATE.
Иначе говоря, ничто не может быть причиной тритмента (факта отправки письма), кроме рандомизации, поэтому потенциальные исходы не зависят от принадлежности к той или иной группе:  $C_0, C_1 \indep T$

In [ ]:
data.query("email==1")["payments"].mean() - data.query("email==0")["payments"].mean()

Проведите АВ-тесты, чтобы оценить различие между группами (`email`) по таргету (`payments`), двумя способами: с помощью `ttest_ind` и модели OLS. Сравните результаты и сделайте выводы.

In [ ]:
from scipy.stats import ttest_ind

group_treated = data.query("email == 1")["payments"]
group_control = data.query("email == 0")["payments"]

ttest_results = ttest_ind(group_treated, group_control, equal_var=False)
print(f"t-statistic: {ttest_results.statistic}, p-value: {ttest_results.pvalue}")

In [ ]:
import statsmodels.formula.api as smf

model = smf.ols(formula='payments ~ email', data=data)
res = model.fit()

print(res.summary().tables[1])

**Вывод:**

1. **t-тест** и **OLS** показали отсутствие значимой разницы между группами (p-value = 0.833).  
2. Коэффициент при `email` (-0.6203) статистически незначим, доверительный интервал включает 0.  
3. **Вывод:** отправка email-напоминаний **не влияет** на платежи клиентов.

Обратим внимание на p-value. Что делать? Вернуться к заказчику, поджав хвост, и сказать, что эксперимент безрезультатен и нужно больше данных?

Но посмотрим на другие признаки

* `credit_limit` &mdash; кредитный лимит клиента до того, как он просрочил платеж;
* `risk_score` &mdash; предполагаемый риск для клиента до момента отправки email.


Добавьте их в качестве признаков и обучите модель


In [ ]:
model_2 = smf.ols(formula='payments ~ email + credit_limit + risk_score', data=data)
res_2 = model_2.fit()
print(res_2.summary().tables[1])

Заметим, что можно извлечь результаты отдельных тестов, например, проверку гипотезы о незначимости коэффициента перед признаком `email`

In [ ]:
result = res_2.t_test('email = 0')
result

Нарисуем causal graph, чтобы визаулизировать влияние величин друг на друга.

*Примечание:* `graphviz` не сохраняет изображения при перемещении ноутбука, поэтому запишем граф в файл с помощью метода `render`

In [ ]:
g = gr.Digraph()

g.node("email", color="gold")
g.edge("credit_limit", "payments")
g.edge("risk_score", "payments")
g.edge("email", "payments")

g.render('emails_covariates', format="png")

In [ ]:
Image(filename='emails_covariates.png')

Рассмотренные признаки *могут объяснить разброс значений таргета*, снизив тем самым его дисперсию. Кроме того, они *не влияют* на принадлежность к группе, тем самым не могут внести смещение.

Итог: **любые признаки, которые влияют на таргет, не влияя при этом на тритмент (принадлежность к А или В группе), можно использовать в модели**. Они называются **ковариатами**. Это помогает снизить дисперсию.

### 2. Использование признаков: нежелательный случай

Загрузим данные `hospital_treatment.csv` из [репозитория](https://github.com/matheusfacure/python-causality-handbook/blob/master/causal-inference-for-the-brave-and-true/data/hospital_treatment.csv).

Две больницы проводят рандомизированное испытание нового препарата для лечения некоторой болезни. **Таргет** &mdash; количество дней госпитализации, хотим сократить эту величину. В одной больнице его назначают 90% пациентам, соответственно 10% пациентов получают плацебо. В другой больнице дают препарат случайным 10% своих пациентам, соответственно 90% получают плацебо. Причем известно, что первая больница обычно получает более тяжелые случаи заболевания.

In [ ]:
DATA_PATH = "/content/drive/MyDrive/colab_data/hospital_treatment.csv"

In [ ]:
hospital = pd.read_csv("/content/hospital_treatment.csv")
hospital.head()

Так как исследование не рандомизированное, то по общим данным смотреть нельзя: обратите внимание на величину коэффициента при параметре `treatment` &mdash; относится пациент к группе, которой назначено лечение (1) или нет (0)

In [ ]:
hosp_1 = smf.ols(formula='days ~ treatment', data=hospital).fit()
print(hosp_1.summary().tables[1])

Но можно посмотреть отдельно по каждой больнице (`hospital`). Постройте две `OLS` на данных каждой из больниц по отдельности.

In [ ]:
hospital_1 = hospital.query("hospital == 1")
hosp_2 = smf.ols(formula='days ~ treatment', data=hospital_1).fit()
print(hosp_2.summary().tables[1])

In [ ]:
hospital_2 = hospital.query("hospital == 0")
hosp_3 = smf.ols(formula='days ~ treatment', data=hospital_2).fit()
print(hosp_3.summary().tables[1])

Поскольку мы рассматриваем каждую больницу отдельно, данных недостаточно, и результаты не являются стат. значимыми.

Но мы знаем, что на результат влияет степень тяжести заболевания, и можем включить ее в модель. Такой признак называется **конфаундером (confounder)**, он влечет как тритмент (принадлежность к группе с назначенным лечением), так и таргет (количество дней госпитализации).

In [ ]:
hosp_4 = smf.ols(formula='days ~ treatment + severity', data=hospital).fit()
print(hosp_4.summary().tables[1])

In [ ]:
g = gr.Digraph()

g.node("treatment", color="gold")
g.edge("severity", "hospital")
g.edge("severity", "days")
g.edge("hospital", "treatment")
g.edge("treatment", "days")

g.render('hospitals_features', format="png")

In [ ]:
Image(filename='hospitals_features.png')

Нужно ли включить больницу в модель? Мы же знаем, что лечение назначается в больницах по-разному, и, кажется, это будет дополнительной информацией.

Однако при фиксировании степени тяжести и тритмента, больница не коррелирует таргетом. Она влечет только тритмент, поэтому *не является* конфаундером. Для снижения дисперсии признак должен быть *хорошей ковариатой таргета*, а не тритмента.

Если мы попробуем его использовать, мы повысим дисперсию. Проверьте это утверждение, построив соответствующую модель.

In [ ]:
hosp_5 = smf.ols(formula='days ~ treatment + severity + C(hospital)', data=hospital).fit()
print(hosp_5.summary().tables[1])

**Выводы:**

1. **Общая модель:**

   - Коэффициент при `treatment`: **+14.15**, значимый (p < 0.05)

   - Без учета конфаундера (`severity`) результат ненадежен

2. **Анализ по больницам:**

   - Коэффициенты при `treatment`: **-10.40** (больница 1) и **-11.41** (больница 2), оба незначимы (p > 0.1)

   - Малые выборки увеличивают неопределенность.

3. **Учет степени тяжести (`severity`):**

   - Коэффициент при `treatment`: **-7.59**, значимый (p < 0.05)

   - Степень тяжести (`severity`): значима (коэффициент **+2.27**, p < 0.05), устраняет смещение.

4. **Включение больницы (`C(hospital)`):**

   - Коэффициент при `treatment`: **-5.09**, становится незначимым (p = 0.149)

   - Больница (`C(hospital)[T.1]`): незначима (p = 0.35)

   - Учет больницы увеличивает дисперсию без пользы.


**Итог:** Использовать модель с признаками `treatment` и `severity`. Больницу исключить — она не улучшает модель.

-----
**Итог:**
* Необходимо добавлять в модель **переменные-ковариаты**, которые влекут таргет (целевую переменную).
* Необходимо добавлять в модель **переменные-конфаундеры**, которые влекут как тритмент, так и таргет.
* Не следует добавлять в модель переменные, которые могут только спрогнозировать тритмент.

### 3. Использование признаков: плохой случай, возникновение смещения

Вернемся к примеру с письмами.

Рассмотрим два дополнительных признака, которые есть в наших данных:
* `opened` &mdash; открыл ли клиент письмо или нет,
* `agreement` &mdash; клиент связался для обсуждения своего долга после получения письма.

Какая из следующих моделей лучше? Только с `email + credit_limit + risk_score` (email_1) или с еще двумя дополнительными признаками (email_2)?

In [ ]:
email_1 = smf.ols(formula='payments ~ email + credit_limit + risk_score', data=data).fit()
print(email_1.summary().tables[1])

In [ ]:
email_2 = smf.ols(formula='payments ~ email + credit_limit + risk_score + opened + agreement', data=data).fit()
print(email_2.summary().tables[1])

Первая модель находит статистически значимые результаты для коэффициента `email`, вторая &mdash; нет. Кажется, раз первая модель смогла найти разницу, то она круче. Но ведь во второй модели мы учли дополнительные факторы, и, может быть, она на самом деле правильная, и тритмент никак не влияет на результат?

---

Опишите, как связаны `opened` и `agreement` с таргетом и тритментом, и ответьте на вопрос, к какому из трех типов признаков (*коварианты, конфаундеры, инструменты*) эти переменные относятся?

**Ответ:**

Модель **email_1** (без `opened` и `agreement`) лучше, потому что в **email_2** эти признаки становятся **инструментами**, влияющими на таргет, и искажают эффект от `email`.  
**`opened`** и **`agreement`** — **инструменты**, связанные с тритментом и таргетом, но не конфаундеры.

*Примечание.* Заметим, что `email` может влиять на таргет не только через `opened`, но и напрямую. Например, клиент увидел письмо, и погасил свой долг, не открывая само письмо.

Проблема заключается в том, что теперь помимо основного пути `email -> payment` у нас еще есть дополнительный путь `email -> opened -> agreement -> payment`. Тем самым часть истинного эффекта влияния тритмента на таргет перетекла на дополнительный путь, который не учтен в коэффициенте линейной регрессии перед `email`. Ведь есть еще и `opened=1`, то вклад в эффект является уже суммой двух коэффициентов линейной регрессии.

## Статистические свойства линейной регрессии

Загрузим датасет с данными по стоимости квартир в Москве (<a href="https://raw.githubusercontent.com/bdemeshev/em301/master/datasets/flats_moscow.txt">источник</a>).

In [ ]:
flats = pd.read_csv("/content/flats_moscow.txt", sep='\t')
flats.head()

Будем рассматривать зависимость цены квартиры от ее площади.

### 1. Обучение простой линейной модели

Сначала обучим линейную регрессию `OLS` предсказания стоимости квартиры по ее площади по аналогии с предыдущими пунктами.

In [ ]:
l_results = smf.ols(formula='price ~ totsp', data=flats).fit()
print(l_results.summary())

Посмотрим на результаты модели, полученные **в предположении гомоскедастичности**.

Можно извлечь результаты отдельных тестов, например, проверка гипотезы о незначимости коэффициента перед признаком `totsp`.

In [ ]:
result = l_results.t_test('totsp = 0')
result

Посмотрите, как зависят остатки обученной модели (используйте функцию `outlier_test()`) от общей площади квартиры

In [ ]:
plt.figure(figsize=(12, 6))
plt.scatter(flats['totsp'], l_results.resid, alpha=0.4)
plt.title('Стоимость квартир в Москве')
plt.xlabel('Общая площадь квартиры, кв.м')
plt.ylabel('Цена квартиры, $1000 (остатки модели)');

По графику явно видна гетероскедастичность.

*Примечание.* Если **хотя бы по одному** признаку, используемому в модели, видна разная дисперсия остатков, то имеет смысл говорить о гетероскедастичности.

### 2. Проверка на гетероскедастичность

*При наличии подозрений на гетероскедастичность рекомендуется сразу использовать устойчивые оценки дисперсии (см. далее), не применяя проверку на гетероскедастичность стат. критериями. Данные критерии также имеют свои ошибки, и последовательное применение нескольких критериев может суммировать вероятности ошибок.*

$\mathsf{H}_0: \gamma_2 = \gamma_3 = ... = \gamma_p ⇔ σ_1^2 = ... = σ_n^2 \Leftrightarrow \,{\large остатки \,\, гомоскедатичны}$

$\mathsf{H}_1: \mathsf{H}_0 \,\,{\large неверна}$

В качестве параметров все методы принимают остатки `resid` и регрессоры (параметры) `model.exog` &mdash; именно в таком порядке

**Критерий Бройша-Пагана**

In [ ]:
test = sms.het_breuschpagan(l_results.resid, l_results.model.exog)

name = ["Lagrange multiplier statistic", "p-value", "f-value", "f p-value"]
pd.Series(test, index=name)

**Критерий Голдфелда-Квандта**

In [ ]:
test = sms.het_goldfeldquandt(l_results.resid, l_results.model.exog)

name = ["F statistic", "p-value", "type"]
pd.Series(test, index=name)

**Критерий Уайта**

In [ ]:
test = sms.het_white(l_results.resid, l_results.model.exog)

name = ["Lagrange multiplier statistic", "p-value", "f-value", "f p-value"]
pd.Series(test, index=name)

### 3. Использование устойчивых оценок дисперсии

Обучите модель `OLS` в условиях гетероскедастичности. Возьмите `cov_type="HC3"` у метода `fit` модели и посмотрите на результат. Интерпретация значений в таблицах аналогична, см. пояснения выше.

In [ ]:
l_results_robust = smf.ols(formula='price ~ totsp', data=flats).fit(cov_type='HC3')

print(l_results_robust.summary())

**Вопрос:** Что изменилось при использовании поправки на гетероскедатичность?

**Ответ:**

При использовании поправки на гетероскедастичность (`cov_type='HC3'`) стандартные ошибки увеличились, что сделало оценки более устойчивыми к изменчивости дисперсии остатков. Коэффициент `totsp` остался значимым, но с более высокой стандартной ошибкой.


### 4. Пример с несколькими признаками

Рассмотрим регрессионную модель $$price = \theta_0 + \theta_1 \cdot totsp + \theta_2 \cdot livesp + \theta_3 \cdot dist$$

Напишите формулу для модели и укажите тип используемой оценки матрицы ковариаций в предположении гетероскедастичности.

In [ ]:
model2 = smf.ols('price ~ totsp + livesp + dist', flats)
results2 = model2.fit(cov_type='HC3')
print(results2.summary())

Как и раньше, можно извлечь результаты отдельных тестов.

In [ ]:
results2.t_test('totsp = 0')

С помощью t-критерия можно проверять одну любую линейную гипотезу. Например, гипотеза о равенстве $\theta_{totsp} = 2\theta_{dist}$.

In [ ]:
results2.t_test('totsp = 2*dist')

Можно использовать  f-критерий для проверки нескольких линейных гипотез.

Например, проверим гипотезу об одновременной незначимости коэффициентов перед признаками `livesp`, `dist`.

In [ ]:
print(results2.f_test("(livesp=0), (dist=0)"))

Можно проверить гипотезу о равенстве $\theta_{totsp} = \theta_{dist}$.

In [ ]:
print(results2.f_test("(livesp=dist)"))

**Выводы:**

1. **Результаты модели**:

   - Модель объясняет 64.4% дисперсии зависимой переменной (R-squared = 0.644)

   - Все коэффициенты значимы (p-значения < 0.05):

     - `totsp`: положительный эффект на цену

     - `livesp`: положительный эффект на цену

     - `dist`: отрицательный эффект на цену

2. **Тесты гипотез**:

   - **Тест для `totsp = 0`**: гипотеза отвергнута (p < 0.05)

   - **Тест для `totsp = 2*dist`**: гипотеза отвергнута (p < 0.05)

   - **Тест для `livesp=0`, `dist=0`**: гипотеза отвергнута (p < 0.05)

   - **Тест для `totsp = dist`**: гипотеза отвергнута (p < 0.05)

3. **Корректировка на гетероскедастичность**:

   - Все стандартные ошибки были скорректированы с помощью метода HC3, что позволяет учесть возможную гетероскедастичность